In [4]:
from vote_system import confront_dataset, filter_pairs
from parsing import parse_stable_code

%load_ext autoreload
%autoreload 2

In [5]:
list_tuple = confront_dataset(["<meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\">", "salut"], 
                 ["<meta name=\"viewport\" content=\"width=device-width>", "salut moi"])

In [6]:
list_tuple

[('<meta name="viewport" content="width=device-width, initial-scale=1.0">',
  '<meta name="viewport" content="width=device-width>',
  0.7777777777777778),
 ('<meta name="viewport" content="width=device-width, initial-scale=1.0">',
  'salut moi',
  0.0),
 ('salut', '<meta name="viewport" content="width=device-width>', 0.0),
 ('salut', 'salut moi', 0.6666666666666666)]

In [7]:
filter_pairs(list_tuple)

[('<meta name="viewport" content="width=device-width, initial-scale=1.0">',
  '<meta name="viewport" content="width=device-width>',
  0.7777777777777778)]

In [35]:
parse_stable_code("datasets/dataset_stable-code_intermediate.json", "airbnb_com")



['#fb_xdm_frame_https { visibility: hidden; }']